# TitaniumCloud search API-s
This notebook guides the user in using the TitaniumCloud search API-s.  
**NOTE:** If pasted into a Python file in the displayed order, all code cells can also work as a Python script.

### Covered API classes
This notebook covers examples for the following API class:
- **AdvancedSearch** (*TCA-0320 - Advanced Search*)
- **ExpressionSearch** (*TCA-0306 - Expression Search with Statistics (Sample Search)*)

### Credentials
Credentials are loaded from a local file instead of being written here in plain text.
To learn how to creat the credentials file, see the **Storing and using the credentials** section in the [README file](./README.md)

### 1. Importing the required classes
First, we will import the required API class from the ticloud module.

In [ ]:
from ReversingLabs.SDK.ticloud import AdvancedSearch, ExpressionSearch

### 2. Loading the credentials
Next, we will load our TitaniumCloud credentials from the local `ticloud_credentials.json` file.
**NOTE: Instead of doing this step, you can paste your credentials while creating the Python object in the following step.**

In [ ]:
import json


CREDENTIALS = json.load(open("ticloud_credentials.json"))
USERNAME = CREDENTIALS.get("username")
PASSWORD = CREDENTIALS.get("password")
USER_AGENT = json.load(open('../user_agent.json'))["user_agent"]

### 3. Using the Advanced Search
The `search` method of the AdvancedSearch class uses a string with space-separated key/value pairs to filter results.
We will use the Advanced Search API to find samples in TitaniumCloud using the following criteria.

#### Available samples with 5 AV scanner results
To fetch all the samples with 5 AV scanner results that are also available for download from TitaniumCloud, do the following:

In [ ]:
advanced_search = AdvancedSearch(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent=USER_AGENT
)

results = advanced_search.search(
    query_string="av-count:5 available:TRUE"
)

print(results.text)

This returned a maximum number of samples for one page of results according to the search criteria that we defined:
- The number of AV scanners that returned results for the samples needs to be 5
- The samples must be available for download 

#### Suspicious samples with a threat level from 0 to 2

In [ ]:
results = advanced_search.search(
    query_string="classification:suspicious threatlevel:[0 TO 2]"
)

print(results.text)

This query returned a maximum number of samples for one page of results according to the search criteria that we defined:
- The samples need to be classified as "suspicious"
- Their threat level needs to be from 0 to 2

### 4. Using the Expression Search
Now we will use the ExpressionSearch class to filter out results and return them using the `search` method of this class.
The search parameters for this class need to be formatted as a list of key/value pair strings.

#### Malicious samples of a specific malware family
What we need now is malicious samples of one of a specific malware family defined by CVE: CVE-2017-11882  
**NOTE: Make sure to set up a date in the "yyyy-MM-dd" format. This date will define the day onwards from which the results will be returned.**

In [ ]:
expression_search = ExpressionSearch(
    host="https://data.reversinglabs.com",
    username=USERNAME,
    password=PASSWORD,
    user_agent=USER_AGENT
)

results = expression_search.search(
    query=[
        "status=MALICIOUS",
        "malware_family=CVE-2017-11882'"
    ],
    date="2023-10-16"
)

print(results.text)

### 5. Paging and additional parameters
Like many other ReversingLabs API-s inside the SDK, the search API-s offer the option to automatically handle paging. Besides only paging, there are also several other options that can be used to refine your search.  
Let us demonstrate those options using the AdvancedSearch object we created.

In [ ]:
search_results = advanced_search.search_aggregated(
    query_string="av-count:5 available:TRUE",
    sorting_criteria="firstseen",
    sorting_order="desc",
    records_per_page=10,
    max_results=50
)

print(search_results)

Our results now returned the following data:
- The number of AV scanners that returned results for the samples is 5
- The samples are available for download
- The 'first seen' date is taken as the sorting criteria
- Results are returned in the descending order
- The maximum number of returned results is 50 with 10 results returned per page

Instead of returning a response object, the paging method return a Python list of results